# How to interpret questions in jupyter notebooks?

How to turn jupyter notebooks in to modules?



In [2]:

try {
    // How to parse a well formatted simple notebook the cheap way?
    var code = (filename) => {
        var co = [];
        var f = fs.readFileSync(filename).toString();
        var re = new RegExp('\\s*"cell_type": "code",\\s*"source":\\s*\\[\\s*([\\s\\S]*?\\$\\$.done[\\s\\S]*?)\\s*\\]\\s*,\s*', 'ig');
        while ((m = re.exec(f)) && co.push(m[1]));
        return JSON.parse('['+co+']').join('') + '\n\n; importNotebook;';
    };
    
    // Execute the notebook
    var importNotebook = ((notebook) => {
        return (($$) => vm.runInThisContext(code(notebook)))
            ({async: () => {}, done: (done) => {}});
    })('How to find answers.ipynb');
} catch (e) {$$.done(e)}



'import ready'

Sucessfully ran a script in it's own context!


[Function: promiseContext]

started notebook How to find answers.ipynb with 6 cells
Sucessfully ran a script in it's own context!
Already imported How to find answers.ipynb
undefined
{ 'How to find answers.ipynb[1]': [Function: getCells],
  'How to find answers.ipynb[2]': [Function: promiseContext],
  'How to find answers.ipynb[3]': [Function: runAllPromises],
  'How to find answers.ipynb[4]': [Function: importNotebook] }


How to find all the notebooks and files?

How to get markdown from all notebooks in {directory}?


In [3]:
// How to flatten and array in javascript?
var flatten = arr => arr.reduce((acc, val) => 
      acc.concat(Array.isArray(val) ? flatten(val) : val), []);

Array.prototype.flatten = function() {return flatten(this)};

// How to escape a string for regex?
function escapeRegExp(str) {
  return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, "\\$&");
}

// How to walk directories and files recursively and synchronously
var walkSync = d => {
    var reg = d.split('*').slice(1).join('[^/]*');
    var dir = d.split('*')[0];
    return flatten(fs.readdirSync(dir)
    .filter(file => path.join(dir, file).match(reg))
    .map(file => fs.statSync(path.join(dir, file)).isDirectory()
        ? walkSync(path.join(dir, file))
        : path.join(dir, file).replace(/\\/g, '/')));
};

// How to test walkSync?
var notebooks = walkSync(path.resolve('**/How to*.ipynb'));
console.log(notebooks);
$$.done(walkSync);


[ '/Users/briancullinan/jupytangular2/notebooks/How to find answers.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to install Docker on Mac.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to install Docker on Windows.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to interpret questions.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to test import.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to use express.ipynb',
  '/Users/briancullinan/jupytangular2/notebooks/How to use simple-imap.ipynb' ]


[Function: walkSync]

How to find questions leading up to jupter cells?

This is a complex question to answer.  Perhaps using IBM Watson language alchemy?

How to find functions in jupyter notebooks?



In [4]:

// TODO: interpret markdown leading up to code results and find the resulting function in the list
// for now, the boring solution is to assume all markdown output is a question?
var cacheCells = ((cells, notebook) => {
    var code = 0, prev = [];
    return cells.reduce((md, c) => {
        if (c.cell_type == 'markdown') {
            prev.push(c.source.join('').trim());
        } else if(c.cell_type == 'code') {
            if(typeof global[notebook+'['+code+']'] == 'function') {
                md.push({
                    id: notebook+'['+code+']',
                    markdown: prev,
                    source: c.source.join('').trim(),
                    function: global[notebook+'['+code+']']
                });
            }
            code++;
            prev = [];
        }
        return md;
    }, []);
});

// How to test search works?
var testNotebook = 'How to test import';
var testCells = [
    {cell_type: 'code', source: []},
    {cell_type: 'markdown', source: ["test for errors line 2"]},
    {cell_type: 'code', source: ["(function (err) { throw error; })"]}
];
global[testNotebook+'[1]'] 
    = eval(testCells[testCells.length-1].source.join('').trim());
var markdownCache = cacheCells(testCells, testNotebook);
if(markdownCache[0].function == global[testNotebook+'[1]']) {
    $$.done(cacheCells);
}



[Function: cacheCells]

How to use fuse.js to implement a simple search?

In [6]:
try {
require('child_process').execSync('npm install fuse.js');
var Fuse = require('fuse.js');
var cellCache = [];
var fuse = new Fuse(cellCache, {
    shouldSort: true, 
    keys: ['markdown'], 
    'id': 'id'
});
var interpretNotebook = (notebook) => {
    return importNotebook(notebook).then(() => 
        getCells(notebook, ['markdown', 'code']))
        .then(cells => {
        consoel.log(cells);
            cellCache.concat(cacheCells(cells, notebook))
        });
};
var interpret = (message) => Promise.resolve(fuse.search(message));

$$.async();
interpretNotebook('How to interpret questions.ipynb').then((results) => {
    console.log(results);
    interpret('test for errors').then(results => {
        console.log(results.length + ' match found ' + results[0]);
        $$.done(interpret);
        return results;
    })
});
} catch (e) {$$.done(e)}

Already imported How to find answers.ipynb,How to interpret questions.ipynb


How to interpret a jupyter {directory}?

In [5]:
var fs = require('fs');
var interpretDirectory = (dirname) => {
    var filenames = walkSync(path.resolve(dirname));
    return Promise.all(filenames.map(notebook => interpretNotebook(notebook)))
        .then(promises => runAllPromises(promises));
}
try {
    // test to make sure it works
    $$.async();
    interpretDirectory('**/How to interpret questions.ipynb')
        .then((results) => {
            console.log(Object.keys(markdownCache));
            $$.done(interpretDirectory);
        }).catch(e => $$.done(e));
}catch(e) {
    $$.done(e);
}


How to answer questions using jupyter notebooks and simple search?

In [7]:
// TODO: function called intend() that errors out if there is not only 1 match with all supplied parameters, must specify the result in the index to not get this error.
